# BRCA1 and BRCA2

This notebook shows how to develop a classifier with embedded tests in Jupyter.

In [ ]:
# !uv pip install -e ../../python

In [ ]:
import pandas as pd
from bioscript import optional_int, optional_str, write_tsv
from bioscript.classifier import GenotypeClassifier
from bioscript.types import VariantCall
from bioscript import assets_dir

In [ ]:
ASSETS_DIR = assets_dir()
RESULT_HEADERS = [
    "participant_id",
    "filename",
    "gene",
    "rsid",
    "chromosome",
    "position",
    "genotype",
    "ref",
    "alt",
    "variant_type",
    "match_type",
]

In [ ]:
def generate_variant_calls(df: pd.DataFrame) -> list[VariantCall]:
    """Generate VariantCall objects from ClinVar DataFrame."""
    vcs: list[VariantCall] = []
    for _, row in df.iterrows():
        vcs.append(
            VariantCall(
                rsid=optional_str(row["rsid"]),
                ref=optional_str(row["ref"]),
                alt=optional_str(row["alt"]),
                chromosome=optional_str(row["chromosome"]),
                position=optional_int(row["position"]),
                gene=optional_str(row.get("gene"), upper=True),
            )
        )
    return vcs

In [ ]:
def get_vcs() -> list[VariantCall]:
    """Load BRCA1 and BRCA2 variant calls from ClinVar TSV files."""
    data_files = [ASSETS_DIR / name for name in ["brca1_clinvar.tsv", "brca2_clinvar.tsv"]]
    dfs = [pd.read_csv(f, sep="\t") for f in data_files]
    df = pd.concat(dfs, ignore_index=True)
    print(f"Loaded {len(df)} variants from BRCA1 and BRCA2")
    return generate_variant_calls(df)

In [ ]:
class BRCAClassifier(GenotypeClassifier):
    def classify(self, matches):
        """Classify BRCA variants and write results to TSV files."""
        if not matches.all_matches:
            print("No variant matches were found.", flush=True)

        # Get categorized matches as report rows
        ref_rows, var_rows, no_rows = matches.categorize_report_rows(
            self.participant_id, self.filename
        )

        if self.debug:
            write_tsv(f"{self.output_basename}_ref.tsv", ref_rows)
            write_tsv(f"{self.output_basename}_no.tsv", no_rows)

        write_tsv(f"{self.output_basename}.tsv", var_rows, headers=RESULT_HEADERS)
        
        # Return variant rows for testing
        return var_rows

In [ ]:
__bioscript__ = {
    "variant_calls": get_vcs,
    "classifier": BRCAClassifier,
    "name": "BRCA",
}

## Tests

Write tests using the test_* function convention:

In [ ]:
from bioscript import VariantFixture
from bioscript.types import MatchList
import os

# Create test fixtures for BRCA1 and BRCA2 variants
fixture = VariantFixture(
    [
        {"rsid": "rs80357336", "chromosome": "17", "position": 43045711},
        {"rsid": "rs886040303", "chromosome": "17", "position": 43045728},
        {"rsid": "rs397509295", "chromosome": "17", "position": 43045729},
        {"rsid": "rs80358650", "chromosome": "13", "position": 32316463},
        {"rsid": "rs397507571", "chromosome": "13", "position": 32316470},
        {"rsid": "rs80358622", "chromosome": "13", "position": 32316497},
    ],
    assembly="GRCh38",
)

In [ ]:
def test_brca1_heterozygous_variants():
    """Test detection of heterozygous BRCA1 variants."""
    # Create test data with heterozygous variants (one alt allele)
    variants = fixture(["GC", "GA", "GT", "GG", "GG", "GG"])
    
    # Create mini variant call list for testing
    test_vcs = [
        VariantCall(rsid="rs80357336", ref="G", alt="C", chromosome="17", position=43045711, gene="BRCA1"),
        VariantCall(rsid="rs886040303", ref="G", alt="A", chromosome="17", position=43045728, gene="BRCA1"),
        VariantCall(rsid="rs397509295", ref="G", alt="T", chromosome="17", position=43045729, gene="BRCA1"),
    ]
    
    matches = MatchList(variant_calls=test_vcs).match_rows(variants)
    classifier = BRCAClassifier(participant_id="TEST_HET", name="BRCA", filename="test.txt")
    result = classifier(matches)
    
    assert len(result) == 3, f"Expected 3 variant rows, got {len(result)}"
    assert all(row["gene"] == "BRCA1" for row in result), "All variants should be BRCA1"
    assert all(row["match_type"] == "VARIANT_CALL" for row in result), "All should be variant calls"
    
    # Cleanup output file
    os.remove("result_BRCA_TEST_HET.tsv")

In [ ]:
def test_brca2_homozygous_variant():
    """Test detection of homozygous BRCA2 variant."""
    # Create test data with one homozygous variant (two alt alleles)
    variants = fixture(["GG", "GG", "GG", "AA", "GG", "GG"])
    
    test_vcs = [
        VariantCall(rsid="rs80358650", ref="G", alt="A", chromosome="13", position=32316463, gene="BRCA2"),
    ]

    matches = MatchList(variant_calls=test_vcs).match_rows(variants)
    classifier = BRCAClassifier(participant_id="TEST_HOM", name="BRCA", filename="test.txt")
    result = classifier(matches)
    
    assert len(result) == 1, f"Expected 1 variant row, got {len(result)}"
    assert result[0]["gene"] == "BRCA2", "Variant should be BRCA2"
    assert result[0]["genotype"] == "AA", "Should be homozygous AA"
    
    # Cleanup output file
    os.remove("result_BRCA_TEST_HOM.tsv")

In [ ]:
def test_no_variants():
    """Test classifier with no matching variants."""
    # All reference genotypes
    variants = fixture(["GG", "GG", "GG", "GG", "GG", "GG"])
    
    test_vcs = [
        VariantCall(rsid="rs80357336", ref="G", alt="C", chromosome="17", position=43045711, gene="BRCA1"),
    ]
    
    matches = MatchList(variant_calls=test_vcs).match_rows(variants)
    classifier = BRCAClassifier(participant_id="TEST_REF", name="BRCA", filename="test.txt")
    result = classifier(matches)
    
    assert len(result) == 0, f"Expected 0 variant rows, got {len(result)}"
    
    # Cleanup output file
    os.remove("result_BRCA_TEST_REF.tsv")

## Run Tests in Jupyter

You can run tests directly in the notebook:

In [ ]:
# Run tests
test_brca1_heterozygous_variants()
test_brca2_homozygous_variant()
test_no_variants()
print("✓ All tests passed!")

## Export to Python Module

Export this notebook to a Python file:

```bash
bioscript export brca_dev.ipynb -o classify_brca.py
```

Or in Python:

```python
from bioscript import export_from_notebook
export_from_notebook("brca_dev.ipynb", "classify_brca.py")
```

In [ ]:
from bioscript import export_from_notebook
export_from_notebook("brca_dev.ipynb", "classify_brca.py")

In [ ]:
!bioscript test classify_brca.py

In [ ]:
from bioscript import export_bioscript_workflow

project = export_bioscript_workflow(
    script_path='./classify_brca.py',
    workflow_name='brca-classifier',
    author='madhava@openmined.org',
    target_dir='./',
    assets={
        "brca1_clinvar.tsv",
        "brca2_clinvar.tsv"
    },
    inputs=[
        {
            'name': 'participants',
            'type': 'List[GenotypeRecord]',
            'description': 'CSV/TSV with participant_id and genotype_file columns',
            'format': 'csv',
            'mapping': {
                'participant_id': 'participant_id',
                'genotype_file': 'genotype_file',
            }
        }
    ],
    outputs=[
        {
            'name': 'classification_result',
            'type': 'File',
            'description': 'BRCA variant classification (aggregated)',
            'format': 'tsv',
            'path': 'result_BRCA.tsv',
        },
    ],
    version="0.1.1",
    description="Classification of BRCA variants using ClinVar reference data for hereditary cancer risk.",
)
project


In [ ]:
from bioscript import export_bioscript_pipeline, PipelineStep, SQLStore

pipeline = export_bioscript_pipeline(
    pipeline_name='brca-classifier',
    target_dir='./brca-classifier',
    inputs={
        'samplesheet': 'List[GenotypeRecord]',
    },
    steps=[
        PipelineStep(
            step_id='brca',
            uses='./',
            with_args={
                'participants': 'inputs.samplesheet',
            },
            publish={
                'classification_result': 'File(result_BRCA.tsv)',
            },
            store={
                'counts_sql': SQLStore(
                    source='classification_result',
                    table_name='brca_{run_id}',
                    destination='SQL()',
                    key_column='participant_id',
                ),
            },
        ),
    ],
    version="0.1.1",
)
pipeline
